In [55]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

import warnings
warnings.filterwarnings('ignore')

In [56]:
data = pd.read_csv('books.csv')

In [57]:
print(data.shape)
print(f'Количество пропусков в данных: {data.isna().sum().sum()}')
print(f'Доля пропусков в данных: {round(data.isna().sum().sum() / (data.shape[0]*data.shape[1]), 2)}')

(3600, 23)
Количество пропусков в данных: 16962
Доля пропусков в данных: 0.2


In [58]:
print(f'Всего объектов у каждого признака (если нет пропусков): {data.shape[0]}')
data.isna().sum().sort_values(ascending=False)

Всего объектов у каждого признака (если нет пропусков): 3600


Разделы комикса           3599
Тип комикса               3599
Художник                  3592
Переводчик                3383
Издательский бренд        2203
Издательство                81
old_price                   80
Тип обложки                 59
cnt_reviews                 50
avg_rating                  50
Тираж                       50
Количество страниц          45
current_price               42
ISBN                        29
Серия                       25
author                      24
Возрастные ограничения      23
Год издания                 18
Вес, г                       2
Размер                       2
ID товара                    2
title                        2
link                         2
dtype: int64

In [59]:
data['cnt_reviews'].fillna('0 оценок', inplace=True)

In [60]:
good_columns = []
bad_columns = []
for column in data.columns:
    all_items = data.shape[0]
    if data.isna().sum().sort_values(ascending=False)[column] / all_items > 0.3:
        bad_columns.append(column)
    else:
        good_columns.append(column)
        
print(f'В признаках {bad_columns} недостаточно объектов, их стоит удалить.')  
print()
print(f'После удаления всех "плохих признаков" остались следующие: {good_columns}')

В признаках ['Издательский бренд', 'Переводчик', 'Художник', 'Тип комикса', 'Разделы комикса'] недостаточно объектов, их стоит удалить.

После удаления всех "плохих признаков" остались следующие: ['link', 'current_price', 'old_price', 'avg_rating', 'cnt_reviews', 'title', 'author', 'ID товара', 'Издательство', 'Серия', 'Год издания', 'ISBN', 'Количество страниц', 'Размер', 'Тип обложки', 'Тираж', 'Вес, г', 'Возрастные ограничения']


In [61]:
data = data[good_columns]
data.dropna(inplace=True)
data = data.reset_index(drop=True)

In [62]:
data['Год издания'] = data['Год издания'].apply(lambda x: str(x).split('.')[0])

data['Год издания'] = data['Год издания'].astype(int)

### Удаляем заполненные, но ненужные признаки

In [63]:
data = data.drop(['ID товара', 'ISBN', 'Серия', 'link', 'old_price', ], axis=1)

In [64]:
print(data.shape)

print(data.columns)

(3260, 13)
Index(['current_price', 'avg_rating', 'cnt_reviews', 'title', 'author',
       'Издательство', 'Год издания', 'Количество страниц', 'Размер',
       'Тип обложки', 'Тираж', 'Вес, г', 'Возрастные ограничения'],
      dtype='object')


In [65]:
data = data.rename(columns={
    'current_price': 'price',
    'Количество страниц': 'pages_cnt',
    'Вес, г': 'weight',
    'Издательство': 'publisher',
    'Год издания': 'publication_year',
    'Тип обложки': 'cover_type',
    'Тираж': 'tirage',
    'Возрастные ограничения': 'reading_age'
    })

In [66]:
data.columns

Index(['price', 'avg_rating', 'cnt_reviews', 'title', 'author', 'publisher',
       'publication_year', 'pages_cnt', 'Размер', 'cover_type', 'tirage',
       'weight', 'reading_age'],
      dtype='object')

### Обработка конкретных признаков:

In [67]:
# надо применить OHE
print(data['cover_type'].value_counts())
print()
print(data['reading_age'].value_counts())
print()
print(data['publication_year'].value_counts())
print()
print(data['publisher'].value_counts())
print()
print(data['author'].value_counts())

cover_type
Твёрдый переплёт                  1728
Мягкий переплёт                   1531
Мягкий заламинированный картон       1
Name: count, dtype: int64

reading_age
16+    2405
12+     680
18+     150
6+       22
0+        3
Name: count, dtype: int64

publication_year
2024    1469
2025     846
2023     571
2022     265
2021      63
2011      12
2012      11
2020       6
2015       5
2018       4
2019       3
2014       2
2010       1
2005       1
2013       1
Name: count, dtype: int64

publisher
АСТ                       1343
Эксмо                     1008
Азбука                     595
Иностранка                 158
Манн, Иванов и Фербер       90
Corpus                      29
Вече                         9
Махаон                       4
Жанры                        3
Альпина Паблишер             3
Проспект                     3
Мартин                       3
Лениздат                     2
Обложка                      2
Агентство Алгоритм ООО       2
КоЛибри                      1
Д

#### Автор российский?

In [68]:
raw_authors = pd.DataFrame(pd.unique(data['author']), columns=['author'])

raw_authors.to_csv('authors.csv')

In [69]:
mapped_authors = pd.read_excel('authors_is_russian_mapping.xlsx')

mapped_authors = mapped_authors.rename(columns={'author': 'author_from_excel'})

In [70]:
authors_merged = mapped_authors.merge(raw_authors, how='inner', right_index=True, left_index=True)

authors_merged = authors_merged.loc[:, ['author', 'is_russian_author']]

In [71]:
data = data.merge(authors_merged, how='left', left_on='author', right_on='author')
data.head()

,price,avg_rating,cnt_reviews,title,author,publisher,publication_year,pages_cnt,Размер,cover_type,tirage,weight,reading_age,is_russian_author
0,312 ₽,4.1,925 оценок,Скорбь Сатаны,Мария Корелли,АСТ,2025,512.0,2.1x11.5x18,Мягкий переплёт,30000.0,250.0,16+,0
1,312 ₽,4.1,1341 оценка,1984 (новый перевод),Джордж Оруэлл,АСТ,2024,320.0,2.4x11.5x18,Мягкий переплёт,30000.0,200.0,16+,0
2,303 ₽,4.2,1288 оценок,Преступление и наказание,Федор Достоевский,АСТ,2025,672.0,2.5x17.5x18,Мягкий переплёт,30000.0,319.0,12+,1
3,284 ₽,4.5,363 оценки,Человек недостойный,Дадзай Осаму,АСТ,2025,192.0,1.5x11.5x18,Мягкий переплёт,25000.0,132.0,16+,0
4,413 ₽,4.2,1269 оценок,Три товарища,Эрих Ремарк,АСТ,2024,480.0,2.5x11.6x18.1,Мягкий переплёт,30000.0,300.0,12+,0


#### Замена значений на "Другой"

In [72]:
def replace_to_other(df, column_name, threshold):

    # Получаем value_counts и фильтруем частые категории
    df[column_name] = df[column_name].astype(str)

    value_counts = df[column_name].value_counts()
    common_categories = value_counts[value_counts >= threshold].index

    df[column_name] = df[column_name].where(df[column_name].isin(common_categories), 'Другой')

In [73]:
replace_to_other(data, 'publication_year', 60)
print(data['publication_year'].value_counts())
print()

replace_to_other(data, 'publisher', 60)
print(data['publisher'].value_counts())
print()

replace_to_other(data, 'author', 50)
print(data['author'].value_counts())
print()

publication_year
2024      1469
2025       846
2023       571
2022       265
2021        63
Другой      46
Name: count, dtype: int64

publisher
АСТ                      1343
Эксмо                    1008
Азбука                    595
Иностранка                158
Манн, Иванов и Фербер      90
Другой                     66
Name: count, dtype: int64

author
Другой               2623
Федор Достоевский     129
Михаил Булгаков        77
Джек Лондон            72
Джейн Остен            68
Эрих Ремарк            67
Джордж Оруэлл          64
Лев Толстой            58
Луиза Мэй Олкотт       52
Николай Гоголь         50
Name: count, dtype: int64



**OHE**

In [74]:
def OHE_features(data):
    categorical_features = []
    for column in data.columns:
        if len(data[column].value_counts()) <= 10:
            #data[column] = data[column].astype(str)
            categorical_features.append(column)
    
    ohe = OneHotEncoder(sparse_output=False)
    ohe_data = ohe.fit_transform(data[categorical_features])
    ohe_columns = ohe.get_feature_names_out(categorical_features)
    ohe_data = pd.DataFrame(ohe_data, columns=ohe_columns)
    
    data = data.drop(categorical_features, axis=1)
    data = pd.concat([data, ohe_data], axis=1)
    
    return data

In [75]:
data = OHE_features(data)

In [76]:
# data = data.drop(columns=['is_russian_author_0']).rename(columns={'is_russian_author_1': 'is_russian_author'})

**Размер**

In [77]:
def split_sizes(data, size_column):
    data['thickness'] = data[size_column].dropna().apply(lambda x: float(x.split('x')[0]))
    data['width'] = data[size_column].dropna().apply(lambda x: float(x.split('x')[1]))
    data['length'] = data[size_column].dropna().apply(lambda x: float(x.split('x')[2]))
    data['volume'] = data['thickness'] * data['width'] * data['length']
    
    data = data.drop([size_column], axis=1)
    
    return data

In [78]:
data = split_sizes(data, 'Размер')

**Некоторые количественные признаки**

In [79]:
# data['old_price'] = data['old_price'].apply(lambda x: x.split(' ')[0])
data['price'] = data['price'].apply(lambda x: x.split(' ')[0])

data['cnt_reviews'] = data['cnt_reviews'].apply(lambda x: x.split()[0]).astype(int)

In [80]:
data.columns

Index(['price', 'avg_rating', 'cnt_reviews', 'title', 'pages_cnt', 'tirage',
       'weight', 'author_Джейн Остен', 'author_Джек Лондон',
       'author_Джордж Оруэлл', 'author_Другой', 'author_Лев Толстой',
       'author_Луиза Мэй Олкотт', 'author_Михаил Булгаков',
       'author_Николай Гоголь', 'author_Федор Достоевский',
       'author_Эрих Ремарк', 'publisher_АСТ', 'publisher_Азбука',
       'publisher_Другой', 'publisher_Иностранка',
       'publisher_Манн, Иванов и Фербер', 'publisher_Эксмо',
       'publication_year_2021', 'publication_year_2022',
       'publication_year_2023', 'publication_year_2024',
       'publication_year_2025', 'publication_year_Другой',
       'cover_type_Мягкий заламинированный картон',
       'cover_type_Мягкий переплёт', 'cover_type_Твёрдый переплёт',
       'reading_age_0+', 'reading_age_12+', 'reading_age_16+',
       'reading_age_18+', 'reading_age_6+', 'is_russian_author_0',
       'is_russian_author_1', 'thickness', 'width', 'length', 'volume']

In [81]:
int_features = ['price', 'cnt_reviews', 
                'pages_cnt', 'tirage']
float_features = ['avg_rating', 'length', 'width', 'thickness', 'weight', 'volume']

In [82]:
data[int_features] = data[int_features].astype(int)
data[float_features] = data[float_features].astype(float)

In [83]:
data.dtypes

price                                          int64
avg_rating                                   float64
cnt_reviews                                    int64
title                                         object
pages_cnt                                      int64
tirage                                         int64
weight                                       float64
author_Джейн Остен                           float64
author_Джек Лондон                           float64
author_Джордж Оруэлл                         float64
author_Другой                                float64
author_Лев Толстой                           float64
author_Луиза Мэй Олкотт                      float64
author_Михаил Булгаков                       float64
author_Николай Гоголь                        float64
author_Федор Достоевский                     float64
author_Эрих Ремарк                           float64
publisher_АСТ                                float64
publisher_Азбука                             f

# Итог:

In [84]:
data.shape

(3260, 43)

In [85]:
data.head()

,price,avg_rating,cnt_reviews,title,pages_cnt,tirage,weight,author_Джейн Остен,author_Джек Лондон,author_Джордж Оруэлл,...,reading_age_12+,reading_age_16+,reading_age_18+,reading_age_6+,is_russian_author_0,is_russian_author_1,thickness,width,length,volume
0,312,4.1,925,Скорбь Сатаны,512,30000,250.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,2.1,11.5,18.0,434.7
1,312,4.1,1341,1984 (новый перевод),320,30000,200.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,2.4,11.5,18.0,496.8
2,303,4.2,1288,Преступление и наказание,672,30000,319.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,2.5,17.5,18.0,787.5
3,284,4.5,363,Человек недостойный,192,25000,132.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.5,11.5,18.0,310.5
4,413,4.2,1269,Три товарища,480,30000,300.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,2.5,11.6,18.1,524.9


In [86]:
data.columns

Index(['price', 'avg_rating', 'cnt_reviews', 'title', 'pages_cnt', 'tirage',
       'weight', 'author_Джейн Остен', 'author_Джек Лондон',
       'author_Джордж Оруэлл', 'author_Другой', 'author_Лев Толстой',
       'author_Луиза Мэй Олкотт', 'author_Михаил Булгаков',
       'author_Николай Гоголь', 'author_Федор Достоевский',
       'author_Эрих Ремарк', 'publisher_АСТ', 'publisher_Азбука',
       'publisher_Другой', 'publisher_Иностранка',
       'publisher_Манн, Иванов и Фербер', 'publisher_Эксмо',
       'publication_year_2021', 'publication_year_2022',
       'publication_year_2023', 'publication_year_2024',
       'publication_year_2025', 'publication_year_Другой',
       'cover_type_Мягкий заламинированный картон',
       'cover_type_Мягкий переплёт', 'cover_type_Твёрдый переплёт',
       'reading_age_0+', 'reading_age_12+', 'reading_age_16+',
       'reading_age_18+', 'reading_age_6+', 'is_russian_author_0',
       'is_russian_author_1', 'thickness', 'width', 'length', 'volume']

### Дропаем лишние OHE, чтобы избежать линейной зависимости

In [87]:
# data = data.drop(['author_Другой', 'publisher_Другой', 'publication_year_Другой', 
#                   'cover_type_Мягкий заламинированный картон', 'reading_age_6+'], axis=1)

In [88]:
data.columns

Index(['price', 'avg_rating', 'cnt_reviews', 'title', 'pages_cnt', 'tirage',
       'weight', 'author_Джейн Остен', 'author_Джек Лондон',
       'author_Джордж Оруэлл', 'author_Другой', 'author_Лев Толстой',
       'author_Луиза Мэй Олкотт', 'author_Михаил Булгаков',
       'author_Николай Гоголь', 'author_Федор Достоевский',
       'author_Эрих Ремарк', 'publisher_АСТ', 'publisher_Азбука',
       'publisher_Другой', 'publisher_Иностранка',
       'publisher_Манн, Иванов и Фербер', 'publisher_Эксмо',
       'publication_year_2021', 'publication_year_2022',
       'publication_year_2023', 'publication_year_2024',
       'publication_year_2025', 'publication_year_Другой',
       'cover_type_Мягкий заламинированный картон',
       'cover_type_Мягкий переплёт', 'cover_type_Твёрдый переплёт',
       'reading_age_0+', 'reading_age_12+', 'reading_age_16+',
       'reading_age_18+', 'reading_age_6+', 'is_russian_author_0',
       'is_russian_author_1', 'thickness', 'width', 'length', 'volume']

In [89]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
price,3260.0,548.441104,365.114493,104.0,299.0,395.0,671.000,4599.00
avg_rating,3260.0,4.364264,0.384091,1.0,4.1,4.3,4.700,5.00
cnt_reviews,3260.0,77.868098,141.659904,1.0,10.0,37.0,89.250,3004.00
pages_cnt,3260.0,475.243865,241.022912,80.0,320.0,416.0,576.000,2720.00
tirage,3260.0,4105.852761,3022.501668,400.0,3000.0,3000.0,4000.000,30000.00
weight,3260.0,385.575153,259.628822,10.0,230.0,310.0,440.000,2840.00
author_Джейн Остен,3260.0,0.020859,0.142934,0.0,0.0,0.0,0.000,1.00
author_Джек Лондон,3260.0,0.022086,0.146985,0.0,0.0,0.0,0.000,1.00
author_Джордж Оруэлл,3260.0,0.019632,0.138753,0.0,0.0,0.0,0.000,1.00
author_Другой,3260.0,0.804601,0.396568,0.0,1.0,1.0,1.000,1.00


# Убираем выбросы

### Убираем выбросы по размеру

In [90]:
data = data[~(data['volume'] == 0)]
data

,price,avg_rating,cnt_reviews,title,pages_cnt,tirage,weight,author_Джейн Остен,author_Джек Лондон,author_Джордж Оруэлл,...,reading_age_12+,reading_age_16+,reading_age_18+,reading_age_6+,is_russian_author_0,is_russian_author_1,thickness,width,length,volume
0,312,4.1,925,Скорбь Сатаны,512,30000,250.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,2.1,11.5,18.0,434.70
1,312,4.1,1341,1984 (новый перевод),320,30000,200.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,2.4,11.5,18.0,496.80
2,303,4.2,1288,Преступление и наказание,672,30000,319.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,2.5,17.5,18.0,787.50
3,284,4.5,363,Человек недостойный,192,25000,132.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.5,11.5,18.0,310.50
4,413,4.2,1269,Три товарища,480,30000,300.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,2.5,11.6,18.1,524.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3255,1241,5.0,3,Махабхарата. Три великих сказания Древней Индии,624,1500,960.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,3.5,16.7,24.0,1402.80
3256,2207,3.7,53,Дракула. Самая полная версия. Коллекционное ил...,448,1500,760.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,2.6,17.0,24.0,1060.80
3257,1103,4.3,42,Время и боги. Дочь короля Эльфландии,784,3000,860.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,3.8,14.5,21.7,1195.67
3258,1011,3.8,29,Роза Мира,704,2000,638.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,3.4,14.0,20.6,980.56


### Убираем выбросы через Isolation Forest

In [91]:
from sklearn.ensemble import IsolationForest

In [92]:
data = data.drop(columns=['title'])

In [93]:
def delete_outliers_iof(data, target):

    df = data.copy()
    X = df.drop([target], axis=1)

    iof = IsolationForest(contamination=0.02, random_state=42)
    iof.fit(X)
    predictions = iof.predict(X)

    return df[predictions >= 0].reset_index(drop=True)

In [94]:
data_after_outliers = delete_outliers_iof(data, 'price')
data_after_outliers

,price,avg_rating,cnt_reviews,pages_cnt,tirage,weight,author_Джейн Остен,author_Джек Лондон,author_Джордж Оруэлл,author_Другой,...,reading_age_12+,reading_age_16+,reading_age_18+,reading_age_6+,is_russian_author_0,is_russian_author_1,thickness,width,length,volume
0,312,4.1,925,512,30000,250.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,2.1,11.5,18.0,434.700
1,312,4.1,1341,320,30000,200.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,2.4,11.5,18.0,496.800
2,284,4.5,363,192,25000,132.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.5,11.5,18.0,310.500
3,339,4.3,872,288,12000,209.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.8,11.5,18.0,372.600
4,284,4.2,3004,320,30000,200.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.6,11.4,17.9,326.496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3185,1241,5.0,3,624,1500,960.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,3.5,16.7,24.0,1402.800
3186,2207,3.7,53,448,1500,760.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,2.6,17.0,24.0,1060.800
3187,1103,4.3,42,784,3000,860.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,3.8,14.5,21.7,1195.670
3188,1011,3.8,29,704,2000,638.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,3.4,14.0,20.6,980.560


In [95]:
data_after_outliers.to_csv('data_after_processing.csv', encoding='utf-8', index=False)